# Домашнее задание к лекции "Основы веб-скрапинга"

## Обязательная часть

Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/).
Функция в качестве параметра должна принимать **список** запросов для поиска (например, `['python', 'анализ данных']`) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:

```
<дата> - <заголовок> - <ссылка на материал>
```

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.


#### Инициализация

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [178]:
URL_domain = 'https://habr.com'
URL_search = 'https://habr.com/ru/search/'

def searching_habr(URL_domain, URL_search, query):
    '''
    Поиск по сайту habr.com
        результаты поиска по каждому запросу, возвращается датафрейм вида:
        <дата> - <заголовок> - <ссылка на материал>
    
    Предполагается работа только с одной (первой) страницей 
    результатов поисковой выдачи для каждого запроса. 
    
    Материалы не дублируются, 
        если они попадали в результаты поиска для нескольких запросов из списка.
    
    Параметры:
        URL_domain - доменное имя сайта (на сайте ссылке хранятся как локальные, к ним надо достроит доменное имя)
        URL_search - поисковая строка для GET
        query - список запросов для поиска (например, ['python', 'анализ данных']) 
    
    '''
    
#   Шаблон результирующего DF
    df_searched = pd.DataFrame(columns=['дата','заголовок','ссылка на материал'])
    
#     Поиск по каждому поисковому запросу
    for qry_el in query:
        params = {'q' : qry_el}
        res = requests.get(URL_search, params = params)
        soup = BeautifulSoup(res.text)

        searched = soup.find_all('article')
    
#         Запись в DF
#         Если запись уже существует, то дополнения не производится

        for el in searched:
#     есть неожиданный блок с другим классом и структурой (текст, ссылка и т.п.)
#     class_=tm-megapost-snippet
#     Его игнорируем, но в целом можно и его распаспарсить
#     из-за этого блока конструкция ниже дает СБОЙ после применения find (элементы разные по структуре)
#     Ошибка:
#         'NoneType' object has no attribute 'text'

            try:
                title = el.find('h2', 'tm-title tm-title_h2').text
                date = el.find('span', 'tm-article-datetime-published').text

                # ссылки в href локальные, надо добавлять домен вручную
                link = el.find('a', 'tm-title__link').get('href')
                link_full = requests.compat.urljoin(URL_domain, link)
            except:
                # Игнорируем те результаты, у которых структура класса другая
                pass
            
#             Если материал уже ранее найден, его не дополняем в DF
            if not df_searched['ссылка на материал'].isin([link_full]).any():
                df_searched.loc[len(df_searched)] = date, title, link_full
                
    return df_searched

# Проверка работы 
query_list=['python', 'анализ данных']
# query_list=['анализ данных'] #конкретно тут был сбой (статья в другом формате)
# query_list=['python']
searching_habr(URL_domain, URL_search, query_list)

,дата,заголовок,ссылка на материал
0,3 мар 2020 в 13:22,В начале этого года Python сместил Java и стал...,https://habr.com/ru/companies/itsumma/news/490...
1,12 апр в 12:57,Ядро планеты Python. Интерактивный учебник,https://habr.com/ru/articles/728568/
2,10 июл в 11:00,"Опенсорс-библиотеки для Python: 40+ вариантов,...",https://habr.com/ru/companies/first/articles/7...
3,26 сен 2022 в 13:54,Java из Python (Kivy) и обратно на Android,https://habr.com/ru/articles/683536/
4,7 июл в 15:27,Как сделать из Python-скрипта исполняемый файл,https://habr.com/ru/companies/southbridge/arti...
5,31 окт 2019 в 10:02,Создатель Python Гвидо ван Россум ушел из Drop...,https://habr.com/ru/news/473926/
6,7 июл в 22:50,Линейная алгебра самым простым языком с добавл...,https://habr.com/ru/articles/746686/
7,21 мар в 16:35,Moscow Python Meetup x YADRO: встречаемся 23 м...,https://habr.com/ru/companies/yadro/news/723806/
8,27 июн в 08:01,Messaging для чайников. Утилизируем все возмож...,https://habr.com/ru/articles/743192/
9,6 мар в 15:53,Как стать Python-разработчиком: сессия вопросо...,https://habr.com/ru/companies/yandex_praktikum...


## Дополнительная часть (необязательная)

Функция из обязательной части задания должна быть расширена следующим образом:
- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента `4` необходимо получить материалы с первых 4 страниц результатов;
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
```
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>
```


In [208]:
URL_domain = 'https://habr.com'
URL_search = 'https://habr.com/ru/search/'

def searching_habr_extended(URL_domain, URL_search, query, page_qty):
    '''
    Поиск по сайту habr.com
        результаты поиска по каждому запросу, возвращается датафрейм вида:
        <дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>
        
    Расширение функции searching_habr():
        - Добавлен  параметр с количеством страниц поисковой выдачи. 
        При передаче в функцию аргумента `4` будут получены материалы с первых 4 страниц результатов
        - добавлены столбцы с полным текстом найденных материалов и количеством лайков
    
    Материалы не дублируются, 
        если они попадали в результаты поиска для нескольких запросов из списка.
    
    Параметры:
        URL_domain - доменное имя сайта (на сайте ссылке хранятся как локальные, к ним надо достроит доменное имя)
        URL_search - поисковая строка для GET
        query - список запросов для поиска (например, ['python', 'анализ данных'])
        page_qty - количество страниц поисковой выдачи    
    '''
    
#   Шаблон результирующего DF
    # количество лайков - такое не найдено на habr. По смыслу подходит "Всего голосов" (внизу статьи)
    df_searched = pd.DataFrame(columns=[
        'дата',
        'заголовок',
        'ссылка на материал', 
        'текст материала', 
        'количество лайков', 
        'поисковый запрос',
        'найдено на странице №'
    ])
    

#     Поиск по каждому поисковому запросу
    for page in range(1, page_qty + 1):

        # счетчик страниц - формируем адрес страницы
        # шаблон на habr: https://habr.com/ru/search/page3/<всякие параметры>
        URL_search_page = URL_search + 'page' + str(page) + '/'
        
        for qry_el in query:
            params = {'q' : qry_el}
            res = requests.get(URL_search_page, params = params)
            
            # проверка существования страницы
            if res.status_code != 200:
                print(f'Страница № {page} не найдена')
                return df_searched
            
            soup = BeautifulSoup(res.text)
            searched = soup.find_all('article')
            
            for el in searched:
                try:
                    title = el.find('h2', 'tm-title tm-title_h2').text
                    date = el.find('span', 'tm-article-datetime-published').text

                    # ссылки в href локальные, надо добавлять домен вручную
                    link = el.find('a', 'tm-title__link').get('href')
                    link_full = requests.compat.urljoin(URL_domain, link)

                    # с лайками есть пара варинтов (отражается либо только добавленные, 
                    # либо текст со статистикой, сколько +, сколько -.)
                    # Оставил вариант +-, как более информативный
                    votes = el.find('div', 
                                    'tm-votes-meter tm-data-icons__item').text
                    
#                     Можно и так, но смысл этой цифры - загадка
#                     votes = el.find('span', 
#                        'tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating').text

#                     Достаем полный текст статьи 

                    article_req = requests.get(link_full)
                    article_soup = BeautifulSoup(article_req.text)
#                     time.sleep(0.3)
                    article_req_txt = article_soup.find('div', id = 'post-content-body').text.strip()

                except:
                    # Игнорируем те результаты, у которых структура класса другая
                    pass

            
#              Запись в DF
#              Если материал уже ранее найден, его не дополняем в DF
                if not df_searched['ссылка на материал'].isin([link_full]).any():
                    df_searched.loc[len(df_searched)] = date, title, link_full, article_req_txt, votes, qry_el, page
                
    return df_searched

# Проверка работы 
query_list=['python', 'анализ данных']
# query_list=['python']
# query_list=['asdgere']
searching_habr_extended(URL_domain, URL_search, query_list, 4)

,дата,заголовок,ссылка на материал,текст материала,количество лайков,поисковый запрос,найдено на странице №
0,3 мар 2020 в 13:22,В начале этого года Python сместил Java и стал...,https://habr.com/ru/companies/itsumma/news/490...,"Согласно отчету RedMonk за январь 2020 года, P...",Всего голосов 31: ↑31 и ↓0 +31,python,1
1,12 апр в 12:57,Ядро планеты Python. Интерактивный учебник,https://habr.com/ru/articles/728568/,"Добрый день! Меня зовут Михаил Емельянов, неда...",Всего голосов 66: ↑66 и ↓0 +66,python,1
2,10 июл в 11:00,"Опенсорс-библиотеки для Python: 40+ вариантов,...",https://habr.com/ru/companies/first/articles/7...,«Удовольствие от программирования на Python до...,Всего голосов 11: ↑11 и ↓0 +11,python,1
3,26 сен 2022 в 13:54,Java из Python (Kivy) и обратно на Android,https://habr.com/ru/articles/683536/,Продолжение статьи C/C++ из Python (ctypes) на...,Всего голосов 4: ↑4 и ↓0 +4,python,1
4,7 июл в 15:27,Как сделать из Python-скрипта исполняемый файл,https://habr.com/ru/companies/southbridge/arti...,Вы изучаете данные и хотите поделиться своим к...,Всего голосов 28: ↑18 и ↓10 +8,python,1
...,...,...,...,...,...,...,...
153,8 сен 2022 в 14:10,Проблемы качества промышленных данных (временн...,https://habr.com/ru/companies/rosatom/articles...,"Привет, Хабр! На связи Юрий Кацер, эксперт по ...",Всего голосов 6: ↑4 и ↓2 +2,анализ данных,4
154,22 июн 2022 в 12:41,Как мы составили Словарь больших данных для те...,https://habr.com/ru/companies/itsumma/articles...,"Я Павел Свиридонов, гуманитарий, который вмест...",Всего голосов 20: ↑18 и ↓2 +16,анализ данных,4
155,8 апр 2013 в 12:52,Открытые репозитории кода по открытым данным и...,https://habr.com/ru/articles/175855/,"Для всех кто работает с открытыми данными, дел...",Всего голосов 31: ↑29 и ↓2 +27,анализ данных,4
156,24 июн 2016 в 23:12,"Достучаться до госорганов или что делать, если...",https://habr.com/ru/companies/infoculture/arti...,источник картинки: southriverrestoration.com/w...,Всего голосов 20: ↑17 и ↓3 +14,анализ данных,4


# Черновики

In [47]:
URL_search = 'https://habr.com/ru/search/'

URL_search = URL_search + 'page' + str(1) + '/'
URL_search

'https://habr.com/ru/search/page1/'

In [77]:
params = {'q' : ['python']}
res = requests.get(URL_search, params = params)
res.status_code
soup = BeautifulSoup(res.text)
soup

<!DOCTYPE html>
<html data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D" lang="ru">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,viewport-fit=cover,maximum-scale=1,user-scalable=0" name="viewport"/>
<meta content="unsafe-url" name="referrer"/>
<title>Результаты поиска по запросу «python» / Хабр</title>
<style>
    /* cyrillic-ext */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveSxf6TF0.woff2) format('woff2');
      unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;
    }

    /* cyrillic */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveQhf6TF0.woff2) format('woff2');
      unicode-range: U+040

In [83]:
title = soup.find('h2', 'tm-title tm-title_h2').text
# votes = soup.find('div', 
#                 'tm-votes-meter tm-data-icons__item').text
votes = soup.find('span', 
                'tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating').text
print(title, votes)

В начале этого года Python сместил Java и стал вторым по популярности языком программирования среди разработчиков +31


In [52]:
page_qty = 4
for page in range(1, page_qty + 1):
    print(page)

1
2
3
4


In [139]:
L_domain = 'https://habr.com'
URL_search = 'https://habr.com/ru/search/'

def test_searching_habr(URL_domain, URL_search, query, searched):
    
#   Шаблон результирующего DF
    df_searched = pd.DataFrame(columns=['дата','заголовок','ссылка на материал'])
    
#     Поиск по каждому поисковому запросу
    for qry_el in query:
        params = {'q' : qry_el}
        res = requests.get(URL_search, params = params)
        soup = BeautifulSoup(res.text)
        print(soup)
        searched = soup.find_all('article')

        print(type(soup),  type(searched))
#         print(len(searched))
#         print(searched[0].prettify())
#         for el in searched[0]:
#             print(el.prettify())
#         for tag in searched.find_all('h2'):
#             print("{0}: {1}".format(tag.name, tag.text))
        
        #             title = el.find('h2', 'tm-title tm-title_h2').text
#             print(title)
#         print(soup.div['tm-articles-list'])
#         searched = soup.find_all('div', class_="tm-articles-list")
#         article_item = searched.find_all('article', class_='tm-articles-list__item').text
#         print(searched)
#         print(article_item)
#         Запись в DF
#         Если запись уже существует, то дополнения не производится
#         for el in searched:
#             title = el.find('h2', 'tm-title tm-title_h2').text
#             date = el.find('span', 'tm-article-datetime-published').text
            
#             # ссылки в href локальные, надо добавлять домен вручную
#             link = el.find('a', 'tm-title__link').get('href')
#             link_full = requests.compat.urljoin(URL_domain, link)
            
# #             Если материал уже ранее найден, его не дополняем в DF
#             if not df_searched['ссылка на материал'].isin([link_full]).any():
#                 df_searched.loc[len(df_searched)] = date, title, link_full
                
    return df_searched, searched

# Проверка работы 
# query_list=['python', 'анализ данных']
# query_list=['анализ данных']
query_list=['python']
test_searching_habr(URL_domain, URL_search, query_list, searched)


<!DOCTYPE html>
<html data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D" lang="ru">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,viewport-fit=cover,maximum-scale=1,user-scalable=0" name="viewport"/>
<meta content="unsafe-url" name="referrer"/>
<title>Результаты поиска по запросу «python» / Хабр</title>
<style>
    /* cyrillic-ext */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveSxf6TF0.woff2) format('woff2');
      unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;
    }

    /* cyrillic */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveQhf6TF0.woff2) format('woff2');
      unicode-range: U+040

(Empty DataFrame
 Columns: [дата, заголовок, ссылка на материал]
 Index: [],
 [<article class="tm-articles-list__item" data-navigatable="" id="490834" tabindex="0"><div class="tm-article-snippet tm-article-snippet"><div class="tm-article-snippet__meta-container"><div class="tm-article-snippet__meta"><span class="tm-user-info tm-article-snippet__author"><a class="tm-user-info__userpic" href="/ru/users/ITSumma/" title="ITSumma"><div class="tm-entity-image"><img alt="" class="tm-entity-image__pic" height="32" src="//habrastorage.org/r/w64/getpro/habr/avatars/b2e/6ed/663/b2e6ed663804346daf634c66b987fe62.png" width="32"/></div></a> <span class="tm-user-info__user tm-user-info__user_appearance-default"><a class="tm-user-info__username" href="/ru/users/ITSumma/">
        ITSumma
        <!-- --></a> <span class="tm-article-datetime-published"><time datetime="2020-03-03T10:22:32.000Z" title="2020-03-03, 13:22">3  мар  2020 в 13:22</time></span></span></span></div> <!-- --></div> <h2 class="tm-

In [186]:
print(type(searched))
print(searched[4].prettify())


<class 'bs4.element.ResultSet'>
<article class="tm-articles-list__item" data-navigatable="" id="473926" tabindex="0">
 <div class="tm-article-snippet tm-article-snippet">
  <div class="tm-article-snippet__meta-container">
   <div class="tm-article-snippet__meta">
    <span class="tm-user-info tm-article-snippet__author">
     <a class="tm-user-info__userpic" href="/ru/users/maybe_elf/" title="maybe_elf">
      <div class="tm-entity-image">
       <img alt="" class="tm-entity-image__pic" height="32" src="//habrastorage.org/r/w64/getpro/habr/avatars/592/11d/4bc/59211d4bcd7f9afdadaef515b2b0706d.jpg" width="32"/>
      </div>
     </a>
     <span class="tm-user-info__user tm-user-info__user_appearance-default">
      <a class="tm-user-info__username" href="/ru/users/maybe_elf/">
       maybe_elf
       <!-- -->
      </a>
      <span class="tm-article-datetime-published">
       <time datetime="2019-10-31T07:02:54.000Z" title="2019-10-31, 10:02">
        31  окт  2019 в 10:02
       </time

In [188]:
print(searched[4].find('h2', 'tm-title tm-title_h2').text)
# print(searched[0].find('div', 'tm-votes-meter tm-data-icons__item').text)
print(searched[4].find('span', 
                       'tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating').text)

Создатель Python Гвидо ван Россум ушел из Dropbox на пенсию
+23


In [164]:
el = searched[0]
title = el.find('h2', 'tm-title tm-title_h2').text
date = el.find('span', 'tm-article-datetime-published').text

# ссылки в href локальные, надо добавлять домен вручную
link = el.find('a', 'tm-title__link').get('href')
link_full = requests.compat.urljoin(URL_domain, link)

print(title, date, link_full)

Ядро планеты Python. Интерактивный учебник 12  апр   в 12:57 https://habr.com/ru/articles/728568/


In [194]:
URL = 'https://habr.com/ru/news/473926/'

res = requests.get(URL)
soup = BeautifulSoup(res.text)
art_txt = soup.find('div', id = 'post-content-body').text.strip()

In [198]:
art_txt = soup.find('div', id = 'post-content-body').text.strip()
art_txt

AttributeError: 'str' object has no attribute 'prettify'

In [209]:
s = '2'
print(type(s))
si = int(s)
print(type(si))

<class 'str'>
<class 'int'>


In [213]:
st = 'afac'
%echo st.replace('a', 'b')

st.replace('a', 'b')


In [214]:
print(f'{123.444:.2f}')

123.44


In [218]:

my_example = {'some_item': 2, 'other_item': 20}
type(my_example)

dict